In [6]:
import numpy as np
import colour
from scipy.optimize import minimize, LinearConstraint, Bounds, NonlinearConstraint

# 获取 BT.2020 色彩空间对象
bt2020 = colour.RGB_COLOURSPACES['ITU-R BT.2020']
srgb = colour.RGB_COLOURSPACES['sRGB']

# 随机采样 RGB（属于 BT.2020 空间的 RGB 向量）
rgb_bt2020 = np.random.rand(1000, 3)
print(rgb_bt2020.shape)

# 将 RGB（BT.2020）转换为 XYZ
xyz_bt2020 = colour.RGB_to_XYZ(
    rgb_bt2020,
    bt2020.whitepoint,
    bt2020.whitepoint,
    bt2020.matrix_RGB_to_XYZ
)

# 将 XYZ 转换为 sRGB 空间的 RGB
rgb_srgb = colour.XYZ_to_RGB(
    xyz_bt2020,
    srgb.whitepoint,
    srgb.whitepoint,
    srgb.matrix_XYZ_to_RGB
)

# 可选：裁剪到 sRGB 显示范围 [0, 1]
rgb_srgb_clipped = np.clip(rgb_srgb, 0, 1)

# 检查有多少点本来就在 sRGB 色域内
in_gamut = np.all((rgb_srgb >= 0) & (rgb_srgb <= 1), axis=1)
print(f"{np.sum(in_gamut)} / {len(rgb_srgb)} colors are in sRGB gamut.")

def objective(M_flat):
    M = M_flat.reshape(3, 3)
    xyz_srgb = (M @ xyz_bt2020.T).T  # Apply M
    rgb_srgb = colour.XYZ_to_RGB(
        xyz_srgb,
        srgb.whitepoint,
        srgb.whitepoint,
        srgb.matrix_XYZ_to_RGB
    )
    lab_target = colour.XYZ_to_Lab(xyz_bt2020)
    lab_result = colour.XYZ_to_Lab(xyz_srgb)
    delta_E = colour.delta_E(lab_target, lab_result, method='CIE 2000')
    return np.mean(delta_E)

def gamut_constraint(M_flat):
    M = M_flat.reshape(3, 3)
    xyz_srgb = (M @ xyz_bt2020.T).T
    rgb_srgb = colour.XYZ_to_RGB(
        xyz_srgb,
        srgb.whitepoint,
        srgb.whitepoint,
        srgb.matrix_XYZ_to_RGB
    )
    # 返回距离越界的最小 margin（负数代表不合法）
    return np.min(np.minimum(rgb_srgb, 1 - rgb_srgb))

# 初始猜测为单位矩阵
M0 = np.eye(3).flatten()

# 非线性约束：所有 RGB 分量必须 ∈ [0, 1]
nonlinear_constraint = NonlinearConstraint(gamut_constraint, -0.01, np.inf)
# 原本的约束条件过于严苛了，这里放宽了标准
print(f"deltaE: {objective(M0)}")

result = minimize(objective, M0, constraints=[nonlinear_constraint])
print(f"Optimization success: {result.success}")
print(f"Optimized matrix M:\n{result.x.reshape(3, 3)}")
print(f"deltaE: {result.fun}")
print(f"Optimization message: {result.message}")

# 获取色彩空间
bt2020 = colour.RGB_COLOURSPACES['ITU-R BT.2020']
srgb = colour.RGB_COLOURSPACES['sRGB']

# 使用您已经优化好的矩阵M
M = result.x.reshape(3, 3)

# 在BT.2020 RGB空间中均匀采样
samples = 100000
np.random.seed(42)  # 设置随机种子以确保结果可复现
rgb_bt2020 = np.random.rand(samples, 3)

# 将RGB(BT.2020)转换为XYZ
xyz_bt2020 = colour.RGB_to_XYZ(
    rgb_bt2020,
    bt2020.whitepoint,
    bt2020.whitepoint,
    bt2020.matrix_RGB_to_XYZ
)

# 计算优化前：直接将BT.2020 XYZ转换为sRGB
rgb_srgb_direct = colour.XYZ_to_RGB(
    xyz_bt2020,
    srgb.whitepoint,
    srgb.whitepoint,
    srgb.matrix_XYZ_to_RGB
)
in_gamut_before = np.all((rgb_srgb_direct >= 0) & (rgb_srgb_direct <= 1), axis=1)
coverage_before = np.sum(in_gamut_before) / samples * 100

# 计算优化后：应用优化矩阵M，然后转换为sRGB
xyz_transformed = (M @ xyz_bt2020.T).T
rgb_srgb_optimized = colour.XYZ_to_RGB(
    xyz_transformed,
    srgb.whitepoint,
    srgb.whitepoint,
    srgb.matrix_XYZ_to_RGB
)
in_gamut_after = np.all((rgb_srgb_optimized >= 0) & (rgb_srgb_optimized <= 1), axis=1)
coverage_after = np.sum(in_gamut_after) / samples * 100

# 输出结果
print(f"采样点数: {samples}")
print(f"优化前色域覆盖率: {coverage_before:.2f}%")
print(f"优化后色域覆盖率: {coverage_after:.2f}%")
print(f"覆盖率变化: {coverage_after - coverage_before:.2f}%")

(1000, 3)
492 / 1000 colors are in sRGB gamut.
deltaE: 0.0
Optimization success: True
Optimized matrix M:
[[ 0.47854502  0.41044372  0.06706907]
 [-0.13498901  1.08885282  0.02455129]
 [ 0.01385766  0.09033064  0.87713752]]
deltaE: 6.105703533027448
Optimization message: Optimization terminated successfully
采样点数: 100000
优化前色域覆盖率: 49.64%
优化后色域覆盖率: 95.93%
覆盖率变化: 46.29%


In [7]:
import numpy as np
import colour

# 获取 BT.2020 色彩空间对象
bt2020 = colour.RGB_COLOURSPACES['ITU-R BT.2020']
srgb = colour.RGB_COLOURSPACES['sRGB']

# 随机采样 RGB（属于 BT.2020 空间的 RGB 向量）
rgb_bt2020 = np.random.rand(1000, 3)
print(rgb_bt2020.shape)

# 将 RGB（BT.2020）转换为 XYZ
xyz_bt2020 = colour.RGB_to_XYZ(
    rgb_bt2020,
    bt2020.whitepoint,
    bt2020.whitepoint,
    bt2020.matrix_RGB_to_XYZ
)


# 将 XYZ 转换为 sRGB 空间的 RGB
rgb_srgb = colour.XYZ_to_RGB(
    xyz_bt2020,
    srgb.whitepoint,
    srgb.whitepoint,
    srgb.matrix_XYZ_to_RGB
)

# 可选：裁剪到 sRGB 显示范围 [0, 1]
rgb_srgb_clipped = np.clip(rgb_srgb, 0, 1)

# 检查有多少点本来就在 sRGB 色域内
in_gamut = np.all((rgb_srgb >= 0) & (rgb_srgb <= 1), axis=1)
print(f"{np.sum(in_gamut)} / {len(rgb_srgb)} colors are in sRGB gamut.")


(1000, 3)
470 / 1000 colors are in sRGB gamut.


In [8]:
from scipy.optimize import minimize, LinearConstraint, Bounds, NonlinearConstraint

def objective(M_flat):
    M = M_flat.reshape(3, 3)
    xyz_srgb = (M @ xyz_bt2020.T).T  # Apply M
    rgb_srgb = colour.XYZ_to_RGB(
        xyz_srgb,
        srgb.whitepoint,
        srgb.whitepoint,
        srgb.matrix_XYZ_to_RGB
    )
    lab_target = colour.XYZ_to_Lab(xyz_bt2020)
    lab_result = colour.XYZ_to_Lab(xyz_srgb)
    delta_E = colour.delta_E(lab_target, lab_result, method='CIE 2000')
    return np.mean(delta_E)

def gamut_constraint(M_flat):
    M = M_flat.reshape(3, 3)
    xyz_srgb = (M @ xyz_bt2020.T).T
    rgb_srgb = colour.XYZ_to_RGB(
        xyz_srgb,
        srgb.whitepoint,
        srgb.whitepoint,
        srgb.matrix_XYZ_to_RGB
    )
    # 返回距离越界的最小 margin（负数代表不合法）
    return np.min(np.minimum(rgb_srgb, 1 - rgb_srgb))

# 初始猜测为单位矩阵
M0 = np.eye(3).flatten()

# 非线性约束：所有 RGB 分量必须 ∈ [0, 1]
nonlinear_constraint = NonlinearConstraint(gamut_constraint, 0, np.inf)

result = minimize(objective, M0, constraints=[nonlinear_constraint])
print(f"Optimization success: {result.success}")
print(f"Optimized matrix M:\n{result.x.reshape(3, 3)}")


Optimization success: False
Optimized matrix M:
[[ 0.46145965  0.42426127  0.07213835]
 [-0.13539576  1.07950558  0.03920639]
 [-0.03053653  0.14380008  0.86727372]]


question 2

这段代码使用了scipy.optimize来进行约束优化,但是没有指定马蹄形比包,所以得到的色域覆盖面积较大但V通道基色是错误的


In [9]:
import numpy as np
from scipy.optimize import minimize

# BT.2020前三基色xy
bt2020_xy = np.array([
    [0.708, 0.292],  # R
    [0.170, 0.797],  # G
    [0.131, 0.046],  # B
])

def polygon_area(points):
    x = points[:, 0]
    y = points[:, 1]
    return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))

def objective(xy):
    # 目标函数：负面积（因为 minimize 是最小化）
    points = np.vstack([bt2020_xy, xy])
    return -polygon_area(points)

# 约束函数，确保点在 xy 色度图的有效区域内
# CIE 1931 xy色度坐标的定义范围是0 <= x,y <= 1 且 x + y <= 1
def constraint1(xy):
    return xy[0]  # x >= 0

def constraint2(xy):
    return xy[1]  # y >= 0

def constraint3(xy):
    return 1 - xy[0] - xy[1]  # x + y <= 1

# 边界条件
bounds = [(0, 1), (0, 1)]

# 约束
constraints = [
    {'type': 'ineq', 'fun': constraint1},
    {'type': 'ineq', 'fun': constraint2},
    {'type': 'ineq', 'fun': constraint3}
]

# 初始猜测
x0 = np.array([0.3, 0.3])

result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=constraints)

if result.success:
    best_xy = result.x
    best_area = -result.fun
    print(f"最佳第四通道基色xy：{best_xy}, 覆盖面积：{best_area}")
else:
    print("优化失败:", result.message)


最佳第四通道基色xy：[ 1.  0.], 覆盖面积：0.33202449999999994


第四通道基色的搜索算法,此处为暴力网格搜索,可以替换为scipy.optimize中的其他优化方法

In [10]:
import numpy as np

# BT.2020前三基色xy
bt2020_xy = np.array([
    [0.708, 0.292],  # R
    [0.170, 0.797],  # G
    [0.131, 0.046],  # B
])

def polygon_area(points):
    # Shoelace formula for polygon area
    x = points[:, 0]
    y = points[:, 1]
    return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))

# 简单网格搜索示例（更高级可用优化算法）
best_area = 0
best_xy = None
for x in np.linspace(0.1, 0.7, 100):
    for y in np.linspace(0.1, 0.8, 100):
        # 简单约束：y < 1 - x (保证点在xy三角形内，不一定完全对应色域边界)
        if y < 1 - x:
            points = np.vstack([bt2020_xy, [x, y]])
            area = polygon_area(points)
            if area > best_area:
                best_area = area
                best_xy = (x, y)

print(f"最佳第四通道基色xy：{best_xy}, 覆盖面积：{best_area}")


最佳第四通道基色xy：(0.69999999999999996, 0.10000000000000001), 覆盖面积：0.26627449999999997
